In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/config.json
/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/merges.txt
/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/README.md
/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/tokenizer.json
/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/vocab.json
/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/tf_model.h5
/kaggle/input/huggingface-roberta-variants/roberta-large-mnli/roberta-large-mnli/pytorch_model.bin
/kaggle/input/huggingface-roberta-variants/xlm-roberta-large/xlm-roberta-large/config.json
/kaggle/input/huggingface-roberta-variants/xlm-roberta-large/xlm-roberta-large/tokenizer.json
/kaggle/input/huggingface-roberta-variants/xlm-roberta-large/xlm-roberta-large/pytorch_model.bin
/kaggle/input/huggingface-roberta-variants/xlm-roberta-large/x

In [2]:
import huggingface_hub
from datetime import datetime
import time
import lightgbm as lgb
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')
import re  
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder

In [3]:
#TRAIN
data = pd.read_csv('../input/commonlitreadabilityprize/train.csv', header = 0)
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv',  header = 0)

data['license'].fillna('None', inplace = True)


data['excerpt'] = [n.replace('{','') for n in data['excerpt']]
data['excerpt'] = [n.replace('}','') for n in data['excerpt']]

data['excerpt'] = [n.replace('(','') for n in data['excerpt']]
data['excerpt'] = [n.replace(')','') for n in data['excerpt']]
data['excerpt'] = [n.replace('.','') for n in data['excerpt']]
data['excerpt'] = [n.replace(',','') for n in data['excerpt']]

#data['excerpt'] = [n.lower() for n in data['excerpt']]

In [4]:
#TEST
test['license'].fillna('None', inplace = True)


test['excerpt'] = [n.replace('{','') for n in test['excerpt']]
test['excerpt'] = [n.replace('}','') for n in test['excerpt']]

test['excerpt'] = [n.replace('(','') for n in test['excerpt']]
test['excerpt'] = [n.replace(')','') for n in test['excerpt']]
test['excerpt'] = [n.replace('.','') for n in test['excerpt']]
test['excerpt'] = [n.replace(',','') for n in test['excerpt']]

#test['excerpt'] = [n.lower() for n in test['excerpt']]

In [5]:
text_data = data['excerpt']

In [6]:
##################################### ROBERTA LARGE

In [7]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer_large = RobertaTokenizer.from_pretrained('../input/huggingface-roberta-variants/roberta-large/roberta-large')
model_large = RobertaModel.from_pretrained('../input/huggingface-roberta-variants/roberta-large/roberta-large',
                                  output_hidden_states = True,# Whether the model returns all hidden-states.
                                  ).to(device)

model_large.eval()

Some weights of the model checkpoint at ../input/huggingface-roberta-variants/roberta-large/roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (d

In [8]:
start_time = datetime.now()
train_data_large = []
with torch.no_grad():
  for i in range(len(text_data)):
    inputs = tokenizer_large(text_data[i], return_tensors="pt").to(device)
    outputs = model_large(**inputs)
    last_hidden_states = outputs.last_hidden_state
    res = torch.mean(last_hidden_states[0], dim=0)
  
    train_data_large.append(res.cpu().data.numpy())
  print(datetime.now() - start_time)

feature_large = [('feature_'+str(i+1)) for i in range(1024)]
df_large = pd.DataFrame(train_data_large, columns = feature_large)

0:01:36.159838


In [9]:
X_large = df_large
y_large = data['target']
#X_train_large, X_val_large, y_train_large, y_val_large = train_test_split(X_large,y_large, test_size = 0.05, random_state = 12)

In [10]:
params_large = {'cat_smooth': 52,
 'colsample_bytree': 0.4,
 'learning_rate': 0.014,
 'max_depth': 9,
 'metric': 'rmse',
 'min_child_samples': 146,
 'n_estimators': 1700,
 'num_leaves': 452,
 'reg_alpha': 0.002281458672564199,
 'reg_lambda': 0.6252458927149529,
 'subsample': 0.8}

model_roberta_large = lgb.LGBMRegressor(**params_large)
model_roberta_large.fit(X_large, y_large)

LGBMRegressor(cat_smooth=52, colsample_bytree=0.4, learning_rate=0.014,
              max_depth=9, metric='rmse', min_child_samples=146,
              n_estimators=1700, num_leaves=452, reg_alpha=0.002281458672564199,
              reg_lambda=0.6252458927149529, subsample=0.8)

In [11]:
#Roberta large test

In [12]:
text_test_large = test['excerpt']

In [13]:
test_data_large = []
with torch.no_grad():
  for i in range(len(text_test_large)):
    inputs = tokenizer_large(text_test_large[i], return_tensors="pt").to(device)
    outputs = model_large(**inputs)
    last_hidden_states = outputs.last_hidden_state
    res = torch.mean(last_hidden_states[0], dim=0)
  
    test_data_large.append(res.cpu().data.numpy())
df_to_test_large = pd.DataFrame(test_data_large, columns = feature_large)
test_id = test['id']

In [14]:
pred_large = model_roberta_large.predict(df_to_test_large)

In [15]:
############################################################### ROBERTA BASE

In [16]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer_base = RobertaTokenizer.from_pretrained('../input/huggingface-roberta-variants/roberta-base/roberta-base')
model_base = RobertaModel.from_pretrained('../input/huggingface-roberta-variants/roberta-base/roberta-base',
                                  output_hidden_states = True,# Whether the model returns all hidden-states.
                                  ).to(device)

model_base.eval()

Some weights of the model checkpoint at ../input/huggingface-roberta-variants/roberta-base/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [17]:
train_data_base = []
with torch.no_grad():
  for i in range(len(text_data)):
    inputs = tokenizer_base(text_data[i], return_tensors="pt").to(device)
    outputs = model_base(**inputs)
    last_hidden_states = outputs.last_hidden_state
    res = torch.mean(last_hidden_states[0], dim=0)
  
    train_data_base.append(res.cpu().data.numpy())

feature_base = [('feature_'+str(i+1)) for i in range(768)]
df_base = pd.DataFrame(train_data_base, columns = feature_base)

In [18]:
X_base = df_base
y_base = data['target']

In [19]:
params_base = {'cat_smooth': 93,
 'colsample_bytree': 0.5,
 'learning_rate': 0.02,
 'max_depth': 6,
 'metric': 'rmse',
 'min_child_samples': 36,
 'n_estimators': 2000,
 'num_leaves': 792,
 'reg_alpha': 0.006198822217531364,
 'reg_lambda': 6.482243311875356,
 'subsample': 0.8}

model_roberta_base = lgb.LGBMRegressor(**params_base)
model_roberta_base.fit(X_base, y_base)

LGBMRegressor(cat_smooth=93, colsample_bytree=0.5, learning_rate=0.02,
              max_depth=6, metric='rmse', min_child_samples=36,
              n_estimators=2000, num_leaves=792, reg_alpha=0.006198822217531364,
              reg_lambda=6.482243311875356, subsample=0.8)

In [20]:
############# ROBERTA base test
text_test_base = text_test_large

In [21]:
test_data_base = []
with torch.no_grad():
  for i in range(len(text_test_base)):
    inputs = tokenizer_base(text_test_base[i], return_tensors="pt").to(device)
    outputs = model_base(**inputs)
    last_hidden_states = outputs.last_hidden_state
    res = torch.mean(last_hidden_states[0], dim=0)
  
    test_data_base.append(res.cpu().data.numpy())
    
df_to_test_base = pd.DataFrame(test_data_base, columns = feature_base)
test_id = test['id']

In [22]:
pred_base = model_roberta_base.predict(df_to_test_base)

In [23]:
###############################################         UNION TEST

In [24]:
pred_union = (pred_base+pred_large)/2

my_submission = pd.DataFrame({'id': test_id, 'target': pred_union})
my_submission.to_csv('submission.csv', index=False)